In [1]:
from ultralytics import YOLO
import cv2
import torch
import torchvision


In [2]:
import math
from sort import *  #It is used to track the objects

In [28]:
model = YOLO('../Yolo-weights/yolov8l.pt')
img = cv2.imread('../Dataset/Library.jpg')
# img = cv2.resize(img, (400,400))
results = model(img, show=True)
cv2.waitKey(0)


0: 640x480 5 persons, 12 chairs, 1 dining table, 1005.8ms
Speed: 5.0ms preprocess, 1005.8ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 480)


-1

##### YOLO with webcam

In [3]:
class_names = []
with open('coco-labels.txt', 'r') as f:
    for label in range(0,80):
        class_names.append(str(f.readline()[:-1]))

In [4]:
class_names[:5]

['person', 'bicycle', 'car', 'motorcycle', 'airplane']

In [ ]:
# # Create 1280*720 mask in canva
# mask = cv2.imread("mask.png")

# Basically, we are trying to count objects once it crosses this limit (it varies).
line_limits = [400, 297, 673, 297]


##### Making a tracker

Max_age defines the waiting time(patience) for the same object to appear on the frame again.

In [5]:
tracker1 = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

#### Vehicle Counter

In [47]:
# cap = cv2.VideoCapture(0)
# # Changing width and height. First parameter in cap.set() is a enumerated propid.
# cap.set(3, 1280)
# cap.set(4, 720)

# Using video file
cap = cv2.VideoCapture('Cars_easy_counting.mp4')

total_count = []
id_set = set()
line_limits = [0, 500, 1280, 500]
while True:
    success, img = cap.read()
    mask_img = img.copy()
    mask = cv2.rectangle(mask_img, (0, 0), (1280, 305), (0, 0, 0), -1)
    imgRegion = cv2.bitwise_and(img, mask)

    model = YOLO('../Yolo-weights/yolov8n.pt')
    results = model(imgRegion, stream=True)

    detections = np.empty((0,5))

    # Bounding Box
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            #cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # Confidence
            conf = math.ceil(box.conf[0]*100)/100

            # Class Name
            cls = int(box.cls[0])
            current_c = class_names[cls]

            if current_c == 'car' or current_c == 'truck' or current_c == 'bus'\
                    or current_c == 'motorbike' and conf > 0.3:
                
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))
                cv2.putText(
                            img,
                            f'Label: {current_c}, Confidence: {conf}',
                            (max(0,x1), max(35, y1-10)),   # Making it 35 so that label won't go outside the window
                            fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                            fontScale = 0.6,
                            color = (255, 0, 255),
                            thickness=2
                        )
    resultsTracker = tracker1.update(detections)

    # Making the tracking line to count cars once it passes the line

    cv2.line(img, (line_limits[0], line_limits[1]), (line_limits[2], line_limits[3]), (0, 0, 255), 5)
    
    for result_tr in resultsTracker:
        x1, y1, x2, y2, id = map(int, result_tr)
        w, h = x2 - x1, y2 - y1
        cv2.rectangle(img, (x1, y1), (x2, y2), (125, 125, 0), 3)
        cv2.putText(img, f'Currently tracking Object ID: {id}', (max(0, x1+5), max(20, y1+25)), fontFace=cv2.FONT_ITALIC,
                    fontScale=0.6, color=(255,255,255), thickness=1)
        id_set.add(id)

        # Center point
        cx, cy = x1+w//2, y1+h//2
        # cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

        # Counting if object(vehicles or any) passes the line limit
        if line_limits[0] < cx < line_limits[2] and line_limits[1]-15 < cy < line_limits[1]+15:
            if total_count.count(id) == 0:
                total_count.append(id)
                cv2.line(img, (line_limits[0], line_limits[1]), (line_limits[2], line_limits[3]), (0, 255, 0), 5)

    counter = len(total_count)
    #counter = len(id_set)        
    cv2.putText(img, f'Count: {counter}', (50, 650), fontFace=cv2.FONT_HERSHEY_COMPLEX,
                   fontScale=1, color=(0,0,125), thickness=2)    

    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()



0: 384x640 1 train, 144.5ms
Speed: 0.7ms preprocess, 144.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 train, 153.9ms
Speed: 0.0ms preprocess, 153.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 train, 144.5ms
Speed: 9.6ms preprocess, 144.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 train, 153.2ms
Speed: 8.0ms preprocess, 153.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 train, 120.6ms
Speed: 0.0ms preprocess, 120.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 train, 116.4ms
Speed: 6.4ms preprocess, 116.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 train, 134.6ms
Speed: 0.0ms preprocess, 134.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 train, 146.3ms
Speed: 0.0ms preprocess, 146.3ms inference, 2.9ms postprocess per image at shape (

#### People Counter

In [46]:
# Using video file
cap = cv2.VideoCapture('People.mp4')

total_count = []
id_set = set()
line_limits = [0, 500, 1280, 500]
while True:
    success, img = cap.read()
    #mask_img = img.copy()
    # mask = cv2.rectangle(mask_img, (0, 0), (1280, 305), (0, 0, 0), -1)
    # imgRegion = cv2.bitwise_and(img, mask)
    img = cv2.resize(img, (1280, 720))
    model = YOLO('../Yolo-weights/yolov8n.pt')
    results = model(img, stream=True)

    detections = np.empty((0,5))

    # Bounding Box
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            #cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # Confidence
            conf = math.ceil(box.conf[0]*100)/100

            # Class Name
            cls = int(box.cls[0])
            current_c = class_names[cls]

            if current_c == 'person' and conf > 0.3:
                
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))
                cv2.putText(
                            img,
                            f'Label: {current_c}, Confidence: {conf}',
                            (max(0,x1), max(35, y1-10)),   # Making it 35 so that label won't go outside the window
                            fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                            fontScale = 0.6,
                            color = (255, 0, 255),
                            thickness=2
                        )
    resultsTracker = tracker1.update(detections)

    # Making the tracking line to count cars once it passes the line

    #cv2.line(img, (line_limits[0], line_limits[1]), (line_limits[2], line_limits[3]), (0, 0, 255), 5)
    
    for result_tr in resultsTracker:
        x1, y1, x2, y2, id = map(int, result_tr)
        w, h = x2 - x1, y2 - y1
        cv2.rectangle(img, (x1, y1), (x2, y2), (125, 125, 0), 3)
        cv2.putText(img, f'Currently tracking Object ID: {id}', (max(0, x1+5), max(20, y1+25)), fontFace=cv2.FONT_ITALIC,
                    fontScale=0.6, color=(255,255,255), thickness=1)
        id_set.add(id)

        # Center point
        cx, cy = x1+w//2, y1+h//2
        # cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

        # Counting if object(vehicles or any) passes the line limit
        # if line_limits[0] < cx < line_limits[2] and line_limits[1]-15 < cy < line_limits[1]+15:
        #     if total_count.count(id) == 0:
        #         total_count.append(id)
        #         #cv2.line(img, (line_limits[0], line_limits[1]), (line_limits[2], line_limits[3]), (0, 255, 0), 5)

    #counter = len(total_count)
    counter = len(id_set)        
    cv2.putText(img, f'Count: {counter}', (50, 650), fontFace=cv2.FONT_HERSHEY_COMPLEX,
                   fontScale=1, color=(0,0,125), thickness=2)    

    cv2.imshow("Image", img)
    # cv2.waitKey(1)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()



0: 384x640 9 persons, 1 train, 1 truck, 8 handbags, 278.8ms
Speed: 4.0ms preprocess, 278.8ms inference, 6.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 1 train, 7 handbags, 268.6ms
Speed: 4.1ms preprocess, 268.6ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 1 train, 8 handbags, 243.9ms
Speed: 8.3ms preprocess, 243.9ms inference, 9.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 1 train, 9 handbags, 269.6ms
Speed: 8.9ms preprocess, 269.6ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 1 train, 8 handbags, 1 tv, 106.2ms
Speed: 5.5ms preprocess, 106.2ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 10 handbags, 1 tv, 124.2ms
Speed: 7.5ms preprocess, 124.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 8 handbags, 1 tv, 110.5ms
Speed: 2.0ms preprocess, 110.5ms infe